In [70]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wang_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wang_\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\wang_\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\wang_\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [77]:
np.random.seed(500)
# insert path to csv file
Corpusi = pd.read_csv(r"D:\Cam_Y2\Group project\Question classifier\test_csv.csv",encoding='latin-1')
Corpusi = pd.read_csv(r"D:\Cam_Y2\Group project\Question classifier\version1.csv",encoding='latin-1',low_memory=False)
print("read complete")

read complete


In [69]:
Corpus = Corpusi.head(1500)
print(Corpus.head(5))

                                                text           label  \
0           When did Beyonce start becoming popular?  conversational   
1  What areas did Beyonce compete in when she was...  conversational   
2  When did Beyonce leave Destiny's Child and bec...  conversational   
3      In what city and state did Beyonce  grow up?   conversational   
4         In which decade did Beyonce become famous?  conversational   

  Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7  
0        NaN        NaN        NaN        NaN        NaN        NaN  
1        NaN        NaN        NaN        NaN        NaN        NaN  
2        NaN        NaN        NaN        NaN        NaN        NaN  
3        NaN        NaN        NaN        NaN        NaN        NaN  
4        NaN        NaN        NaN        NaN        NaN        NaN  


In [78]:
dtype = {'text': str,'label': str}
dirty_data = Corpusi
row_count = len(dirty_data.index)
med_data = dirty_data[dirty_data["label"] == 'medical']
dirty_data = dirty_data.sample(n = 1500)
dirty_data = dirty_data[dirty_data["label"] == 'conversational']
new_data = pd.concat([dirty_data,med_data])
Corpus = new_data.sample(frac = 1).reset_index(drop=True)
is_list = type(Corpus['text'].head().iloc[0])
print(is_list)
print(Corpus.head(5))

<class 'str'>
                                                text           label  \
0  When was a second public service held for Prin...  conversational   
1               What is the name of Queen's drummer?  conversational   
2  How many games has Everton played against Asto...  conversational   
3                    To what is Sanskrit restricted?  conversational   
4  What is one reasons Shakespeare is a good plac...  conversational   

  Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7  
0        NaN        NaN        NaN        NaN        NaN        NaN  
1        NaN        NaN        NaN        NaN        NaN        NaN  
2        NaN        NaN        NaN        NaN        NaN        NaN  
3        NaN        NaN        NaN        NaN        NaN        NaN  
4        NaN        NaN        NaN        NaN        NaN        NaN  


In [79]:



# Step - a : Remove blank rows if any.
Corpus['text'].dropna(inplace=True)

# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['text'] = [entry.lower() for entry in Corpus['text']]
# (Alex) Website said: Corpus['text'] = [entry.str.lower() for entry in Corpus['text']]

# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]
# (Alex) 
# I'm not really sure what is going on here, but the imported data is already in tocken form. The series Corpus['text'] 
# contains lists of words, which is essentially the tockenised form. Supposedly, the author of the webpage got a string 
# as each entry in Corpus['text'], thus he used ".lower()" on the entry, then try to tockenise it.
# Finally, the above prosedure does not take signifigantly more time, plus this should be done at configure time, so it should be fine

# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun

print("step a-c complete")

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

print(Corpus['text_final'].head(5))

step a-c complete
0    ['second', 'public', 'service', 'hold', 'princ...
1                         ['name', 'queen', 'drummer']
2    ['many', 'game', 'everton', 'play', 'aston', '...
3                             ['sanskrit', 'restrict']
4    ['one', 'reason', 'shakespeare', 'good', 'plac...
Name: text_final, dtype: object


In [80]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)

In [81]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
print(Corpus['text_final'].head(5))

0    ['second', 'public', 'service', 'hold', 'princ...
1                         ['name', 'queen', 'drummer']
2    ['many', 'game', 'everton', 'play', 'aston', '...
3                             ['sanskrit', 'restrict']
4    ['one', 'reason', 'shakespeare', 'good', 'plac...
Name: text_final, dtype: object


In [82]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [84]:
#print(Tfidf_vect.vocabulary_)
#print(Train_X_Tfidf)

In [85]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  92.19269102990033
